In [1]:
from scipy import integrate
from scipy.optimize import fsolve
from math import pi, e, cos, sin
import numpy as np
from IPython.display import display, Markdown
import pandas as pd

## Find the effective thickness of the photoevaporation flow

To calculate $h_0$  we use the equation for [r](https://github.com/will-henney/globule-seminario/issues/5#issuecomment-1438984643) and $rho$ $$\frac{\rho}{\rho_0}=e^{\frac{1-M^2}{2}}$$ and $$\frac{r}{r_0}=M^{-1/\alpha}e^{\frac{M^2-1}{2 \alpha}}$$
hence
$$\frac{dr}{dM}=r_0\Big( e^{\frac{M^2-1}{2\alpha}}\Big(-\frac{M^{-1/\alpha-1}}{\alpha}+\frac{M}{\alpha M^{1/\alpha}}\Big)\Big)=\frac{r_0}{\alpha}e^{\frac{M^2-1}{2\alpha}}\Big(M^{\frac{\alpha-1}{\alpha}}-M^{-\frac{\alpha+1}{\alpha}}\Big)$$
therefore
$$h_0=\int_0^\infty \Big(\frac{n(r)}{n_0}\Big)^2 dr=\int_1^\infty \frac{r_0}{\alpha} (exp(1-M^2))^{1-\frac{1}{2\alpha}}\Big(M^{\frac{\alpha-1}{\alpha}}-M^{-\frac{\alpha+1}{\alpha}}\Big) dM.$$

For $\alpha=2$ we have $$h_0=r_0\int_1^\infty \frac{exp(\frac{3}{4}(1-M^2))}{2}(M^{1/2}-M^{-3/4})dM$$

In [2]:
def alpha_values(alpha): #integrate in function of alpha
    function = lambda x: ((e**(1-x**2))**(1-1/(2*alpha)))*(x**((alpha-1)/alpha)-x**(-(alpha+1)/alpha))/alpha
    return(function)
 
h_0, err = integrate.quad(alpha_values(2), 1, np.inf) #alpha=2 spherical case
display(Markdown(rf'The values of $h_0/r_0$ for $\alpha=2$ is'))
display(Markdown(rf'$h_0/r_0=$'f'{h_0}'))

The values of $h_0/r_0$ for $\alpha=2$ is

$h_0/r_0=$0.12149200100709526

Now for the cylindrical case $\alpha=1$ and we have $$h_0=r_0\int_1^\infty \Big(exp(1-M^2)\big)^{1/2}\Big(1-M^{-2}\Big) dM$$

In [3]:
h_0, err = integrate.quad(alpha_values(1), 1, np.inf) #alpha=1 cylindrical case
display(Markdown(rf'The values of $h_0/r_0$ for $\alpha=1$ is'))
display(Markdown(rf'$h_0/r_0=$'f'{h_0}'))

The values of $h_0/r_0$ for $\alpha=1$ is

$h_0/r_0=$0.311359084837597

## Properties of the shocked shell for photoevaporation flow that interacts with an ambient pressure

We investigate a range of pressure contrasts $P / P_0$ between the ambient gas and the base of the photoevaporatoin flow. 

In [4]:
P_P_0_1 = [i+2 for i in range(14)]  #P/P_0 decrease to 0
Extr=[1/0.9,1/0.8,1/0.7,1/0.6]      #case when P/P_0 is near of 1
P_P_0 = 1 / np.array(Extr + P_P_0_1)
pd.DataFrame(P_P_0)#.style.format(precision=2)

,0
0,0.900000
1,0.800000
2,0.700000
3,0.600000
4,0.500000
5,0.333333
6,0.250000
7,0.200000
8,0.166667
9,0.142857


Find the Mach number of the flow at radius where the shocked shell will form, assuming that the flow ram pressure is equal to the ambient pressure. 

Now we use the pressure equation to find the mach number

$$\frac{P}{P_0}=\frac{1+M^2}{2}exp\Big(\frac{1-M^2}{2}\Big)$$

In [5]:
def func_to_find_M0(x):
    """Should be called with vector argument of same length as P_P_0"""
    return ((1+x**2)/2)*(e**((1-x**2)/2)) - P_P_0 #equation to find the root

M0 = fsolve(func_to_find_M0, 1.5 * np.ones_like(P_P_0)) 
#we find the mach number to differents ratio of P/P_0
M0

array([1.43653166, 1.62750626, 1.78737193, 1.93722551, 2.0872695 ,
       2.36189814, 2.52690899, 2.64359919, 2.73319847, 2.80555349,
       2.86601589, 2.91780729, 2.96301204, 3.00305324, 3.03894401,
       3.07143012, 3.10107584, 3.12831796])

Find the other parameters of the shocked shell: $r_1 / r_0$, $n_1 / n_0$ (density just before shock). 

The equations for $r_1$ and $\rho_1$ are $$\frac{\rho_1}{\rho_0}=e^{\frac{1-M^2}{2}}$$ and $$\frac{r_1}{r_0}=M^{-1/\alpha}e^{\frac{M^2-1}{2 \alpha}}$$

In [6]:
alpha=2      #spherical case
Table1 = pd.DataFrame()
Table1[rf'$P_1/P_0$'] = P_P_0
Table1[rf'$M_0$'] = M0
Table1[rf'$\rho_1/\rho_0$'] = e**((1-M0**2)/2)
Table1[rf'$r_1/r_0$'] = (M0**(-1/alpha))*e**((M0**2-1)/(2*alpha))
Table1#.style.format(precision = 2)

,$P_1/P_0$,$M_0$,$\rho_1/\rho_0$,$r_1/r_0$
0,0.900000,1.436532,0.587540,1.088488
1,0.800000,1.627506,0.438503,1.183729
2,0.700000,1.787372,0.333755,1.294729
3,0.600000,1.937226,0.252480,1.429868
4,0.500000,2.087270,0.186682,1.601988
5,0.333333,2.361898,0.101339,2.043998
6,0.250000,2.526909,0.067702,2.417705
7,0.200000,2.643599,0.050071,2.748578
8,0.166667,2.733198,0.039353,3.049133
9,0.142857,2.805553,0.032207,3.326705


Find the properties of the shocked shell itself (density and thickness): $n_2 / n_0$, $h_2 / h_0$. 

For density we have that in the shock shell  from $\rho_1 M_1=\rho_2M_2$, where $M_2=M_1^{-1}$, therefore
$$\frac{\rho_2}{\rho_0}=(\rho_2/\rho_1)(\rho_1/\rho_0)=M_1^2(\rho_1/\rho_0)$$

and for $h_2/h_0$ we have that in  the shock shell $h_2/r_1=\frac{3}{4M_1^2}$ hence
$$\frac{h_2}{r_0}=(h_2/r_1)(r_1/r_0)=\frac{3}{4M_1^2}\frac{r_1}{r_0}$$

In [71]:
Table2=pd.DataFrame()
Table2[rf'$P_1/P_0$'] = P_P_0
Table2[rf'$M_0$'] = M0
Table2[rf'$\rho_2/\rho_0$'] = (M0**2)*np.array(Table1.loc[:,[rf'$\rho_1/\rho_0$'][0]]) #with data of Table1
Table2[rf'$h_2/r_0$'] = (3/(4*M0**2))*np.array(Table1.loc[:,[rf'$r_1/r_0$'][0]])
Table2

,$P_1/P_0$,$M_0$,$\rho_2/\rho_0$,$h_2/r_0$
0,0.900000,1.436532,1.212460,0.395599
1,0.800000,1.627506,1.161497,0.335172
2,0.700000,1.787372,1.066245,0.303956
3,0.600000,1.937226,0.947520,0.285757
4,0.500000,2.087270,0.813318,0.275780
5,0.333333,2.361898,0.565327,0.274802
6,0.250000,2.526909,0.432298,0.283978
7,0.200000,2.643599,0.349929,0.294970
8,0.166667,2.733198,0.293980,0.306123
9,0.142857,2.805553,0.253507,0.316985


In [2]:
#IGNORE
alpha=2
    
h_R_0 = [(i**(-1/alpha))*(3/(4*i**2))*e**((i**2-1)/(2*alpha)) for i in Roots]
rho2_rho0 = [(i**2)*e**((1-i**2)/2) for i in Roots]

Rec_h2n2 = [(3/4)*(1/0.12)*(i**(2-1/alpha))*(e**((i**2-1)/(2*alpha)))*(e**(i**2-1)) for i in Roots]


T3 = pd.DataFrame(data = [[1/P_P_0[i],Roots[i],rho2_rho0[i],h_R_0[i],Rec_h2n2[i]] for i in range(len(P_P_0))],
                columns = [rf'$P/P_0$',rf'$M_1$', rf'$\rho_2/\rho_0$',rf'$h/r_0$',rf'($\alpha_\beta h_2 n_2)/(\alpha_\beta h_0 n_0$)'])
T3

,$P/P_0$,$M_1$,$\rho_2/\rho_0$,$h/r_0$,($\alpha_\beta h_2 n_2)/(\alpha_\beta h_0 n_0$)
0,0.900000,[1.43653166229611],[1.212460386727212],[0.3955985401577511],[40.66866708341336]
1,0.800000,[1.62750625745831],[1.161496861145731],[0.335172394947937],[101.91362555071417]
2,0.700000,[1.7873719314700518],[1.066245374672153],[0.30395569363387],[232.0781132269163]
3,0.600000,[1.9372255119437625],[0.9475195204739406],[0.28575702167852984],[526.1151053896526]
4,0.500000,[2.0872695034502167],[0.8133176911491646],[0.2757803777585336],[1251.6701644116563]
5,0.333333,[2.3618981411410918],[0.565327410056532],[0.2748017508441885],[6939.488790260166]
6,0.250000,[2.5269089927774195],[0.43229765955875116],[0.2839784731235524],[21050.1508783226]
7,0.200000,[2.643599193146395],[0.3499287529591649],[0.29497012544141],[47885.32219070201]
8,0.166667,[2.733198469715219],[0.2939804856009608],[0.3061225050545589],[91927.69226537192]
9,0.142857,[2.8055534914820472],[0.25350708403438516],[0.31698476038720674],[157770.34148863688]


## Emission measure of shocked shell

*Calculate the relative emission measure of the shell and the photoevaporation flow*


*Also calculate the peak surface brightness of the shell and photoevaporation flow. For this you will need to estimate the line of sight depths $\ell$*

*Make a second table showing these quantities. Alternatively, show fewer significant figures and see if everything will fit in a single table*